NameError: name 'vp' is not defined

In [1]:
from datetime import timedelta
from SPICE.ephemeris import ephemeris
import spiceypy as sp
from datetime import datetime
from gravsystem import gravsystem
import vpython as vp
from config.parse_config import parse_config

def timeflow(real_duration, animation_duration, frame_rate):
    total_frames = animation_duration * frame_rate
    real_frame_period = real_duration / total_frames
    ss.draw()
    for _ in range(total_frames):
        vp.rate(frame_rate)
        ss.update(real_frame_period)
        ss.draw()
        
class dotdict(dict):
    """modified from https://stackoverflow.com/questions/2352181"""
    """dot.notation access to dictionary attributes"""
    def __getattr__(*args): 
        val = dict.get(*args)
        return dotdict(val) if type(val) is dict else val
  
    def __getitem__(*args):
        val = dict.get(*args)
        return dotdict(val) if type(val) is dict else val
        
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__
    
scale = 1e-6
import numpy as np
V = vp.vector
arr = np.array
def draw_solution(aE, bE, dt, f2, v1, v2, Rinv):
    vp.sphere(pos=V(*(p2*scale)), radius=5, color=vp.color.purple)
    vp.arrow(pos=V(*(p1*scale)), axis=V(*(v1)), color=V(1,0,0), radius=3, shaftwidth=0.2)
    vp.arrow(pos=V(*(p2*scale)), axis=V(*(v2)), color=V(0,0,1), radius=3, shaftwidth=0.2)
    vp.sphere(pos=V(*(f2*scale)), radius=1, color=vp.color.orange)
    circ = vp.shapes.circle(pos=(0,0), radius=0.2)
    path = vp.paths.ellipse(pos=V(*(f2/2*scale)), 
                             width=aE*scale, height=bE*scale)
    orbit = vp.extrusion(path=path, shape=circ, axis=V(*f2))   
    up = Rinv(arr([0,0,1]))
    orbit.up = V(*up)

def convert(dateObj):
    return sp.str2et(dateObj.strftime("%Y %b %d %H:%M:%S").lower())


config = parse_config('C:/Users/HP/GravAssist/src/config/config.json')
now = datetime.today()
ephemeris = ephemeris(sp, config)
ss = gravsystem(config, ephemeris, now)
scene = vp.canvas(title='3D scene')
timeflow(timedelta(days=0), 1, 10)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [4]:
timeflow(timedelta(days=90), 1, 10)

In [2]:
import numpy as np
from lambert import lambert_transfer

T = timedelta(days=900)
GM = ss.body_list['SUN'].const('GM', 1)
p1 = ss.body_list['EARTH'].state(ss.time)[:3]
p2 = ss.body_list['JUPITER'].state(ss.time+T)[:3]

Rsoi = {'EARTH': 0.929e6, 'MERCURY':0.112e6, 'VENUS':0.616e6,
       'MARS':0.578e6, 'JUPITER':48.2e6, 'SATURN':54.5e6,
       'URANUS':51.9e6, 'NEPTUNE':86.8e6}

v1 = ss.body_list['EARTH'].state(ss.time)[3:]
p1_ = p1 + v1/np.linalg.norm(v1) * Rsoi['EARTH']

solutions = lambert_transfer(p1_, p2, ss.time, T, GM)
solution = solutions[0]
#s2 = solutions[1]

#TE = timedelta(seconds=((4 * np.pi**2 * solution['aE']**3 / GM) ** 0.5)[0])
#print(solution['dt'])
draw_solution(**solution)

vp.sphere(pos=V(*(p2*scale)), color=vp.color.purple, size=V(8,8,8))

pf = np.concatenate((p1_, solution['v1']))
#for i in range(100):
pf = sp.prop2b(GM, pf, T.total_seconds())
vp.sphere(pos=V(*(pf[:3]*scale)), color=V(1, 1, 0), size=V(10,10,10))



C:\Users\HP\GravAssist\src\lambert.py:76: RuntimeWarning: invalid value encountered in arccos
  th = np.arccos(np.sum(r1E * r2E) / norm(r1E) / norm(r2E))


In [5]:
vp.sphere(pos=V(*(ss.body_list['MARS'].state(ss.time+T)[:3]*scale)), size=V(8,8,8), color=vp.color.purple)
solutions = lambert_transfer(p1, p2, ss.time, T, GM)
sol = solutions[0]
draw_solution(**sol)

In [18]:
import time
from imp import reload
import lambert
reload(lambert)

start = time.time()
for _ in range(10):
    solutions = lambert.lambert_transfer(p1, p2, ss.time, T, GM)
end = time.time()
print(end-start)

3.686765193939209


In [20]:
import copy
RSOI = {'EARTH': 0.929e6, 'MERCURY':0.112e6, 'VENUS':0.616e6,
       'MARS':0.578e6, 'JUPITER':48.2e6, 'SATURN':54.5e6,
       'URANUS':51.9e6, 'NEPTUNE':86.8e6}

def entrancePt(Rsoi, th1, th2):
    ept = np.array([Rsoi, 0, 0]).T
    Mry = np.array([[np.cos(th1), 0, np.sin(th1)], [0, 1, 0], [-np.sin(th1), 0, np.cos(th1)]])
    Mrz = np.array([[np.cos(th2), -np.sin(th2), 0], [np.sin(th2), np.cos(th2), 0],[0, 0, 1]])
    ept = np.matmul(Mry, ept)
    ept = np.matmul(Mrz, ept)
    return ept


#in planet's reference frame
def swingby(state0_, GM, Rsoi, time_precision=60):
    statef = copy.deepcopy(state0_)
    steps = 0
    while np.linalg.norm(statef[:3])/Rsoi < 1.01:
        statef = sp.prop2b(GM, statef, time_precision)
        steps += 1
    return statef, time_precsion * steps

T = timedelta(days=100)
GM = ss.body_list['SUN'].const('GM', 1)
p1 = ss.body_list['EARTH'].state(ss.time)[:3]
state2 = ss.body_list['MARS'].state(ss.time+T)
p2 = state2[:3]
vm = state2[3:]

import lambert
lmb = lambert.lambert_transfer(p1, p2, ss.time, T, GM)[1]
v_ent = lmb['v2'] - vm

draw_solution(**lmb)

"""
for i in range(20):
    for j in range(10):
        th1 , th2 = i * np.pi /10, j*np.pi / 10
        ent = entrancePt(RSOI['MARS'], th1, th2)
        state0 = np.concatenate((ent, v_ent))
        statef = sp.prop2b(ss.body_list['MARS'].const('GM', 1), state0, 1000)
        if np.linalg.norm(state0) > np.linalg.norm(statef):
            vp.sphere(pos=V(*((ent*100 + p2)*scale)), size=V(2, 2, 2), color=vp.color.green)
            print(i, j)
"""
th1 , th2 = 0, -2
ent = entrancePt(RSOI['MARS'], th1, th2)
vp.arrow(pos=V(*(p2*scale)), axis=V(*entrancePt(1, 0, 0)), shaft_width=0.1)

vp.arrow(pos=V(*(p2*scale)), axis=V(*entrancePt(1, np.pi/2, 0)), shaft_width=0.1, color=vp.color.green)
vp.arrow(pos=V(*(p2*scale)), axis=V(*entrancePt(1, 0, np.pi/2)), shaft_width=0.1, color=vp.color.red)

statef = np.concatenate((ent, v_ent))
step = timedelta(minutes=100)
cnt = 0
while np.linalg.norm(statef[:3])/RSOI['MARS'] < 1.01:
        statef = sp.prop2b(ss.body_list['MARS'].const('GM', 1), statef, step.total_seconds())
        pm = ss.body_list['MARS'].state(ss.time+T + cnt * step)[:3]
        x= statef[:3]+pm
        
        
        vp.sphere(pos=V(*()*scale)), size=V(2, 2, 2), color=vp.color.green)
        cnt += 1

        
